# Imports

In [15]:
from systemtools.hayj import *
from systemtools.location import *
from systemtools.basics import *
from systemtools.file import *
from systemtools.printer import *
from twinews.utils import *
from twinews.evaluation.utils import *
from twinews.models.ranking import *
from dataviztools.bokehutils import *

In [2]:
import bokeh
from bokeh.plotting import figure, output_notebook, show, ColumnDataSource
from bokeh.models import Grid, Legend, LegendItem
from bokeh.layouts import gridplot
from bokeh.io import export_svgs, export_png
output_notebook()

Loading BokehJS ...

In [3]:
def getModel(row):
    model = row['model']
    if model == 'combin':
        model = row['models'][0][:-6] + " " + row['models'][1][:-6]
    return model

In [4]:
def bokehPointsSetup(fig, amount, shapeAmount=5, paletteIndex=0):
    global palettes
    # We define all shapes:
    def __shapes(p):
        return [p.triangle, p.circle, p.diamond, p.square, p.inverted_triangle][:shapeAmount]
    shapes = __shapes(p)
    # We define all colors:
    colors = palettes[paletteIndex]
    # We generate all combinasons
    dots = []
    for i in range(len(shapes)):
        for u in range(len(colors)):
            f = (u + i) % 5
            s = u
            dots.append((shapes[s], colors[f]))
    return dots[:amount]

In [5]:
def isModels(row, *args):
    if 'models' not in row:
        return False
    else:
        modelsRepr = str(row['models'])
        for arg in args:
            if arg not in modelsRepr:
                return False
        return True

In [6]:
def getColorKwargs(color):
    return {'fill_color': color, 'line_color': color}

In [7]:
# https://learnui.design/tools/data-color-picker.html#palette
palettes = \
[
    ['#488f31', '#a7c162', '#fff59f', '#f49e5c', '#de425b'],
    ['#003f5c', '#58508d', '#bc5090', '#ff6361', '#ffa600'],
    ['#009b95', '#007acc', '#bc5090', '#aaaaaa', '#ff8f8f', '#ef9a32'],
    ['#a31430', '#0071bd', '#02bebf', '#df7d00', '#017f01'],
]

# Getting data

In [ ]:
models2 = twinewsGet\
(
    splitVersion=2,
    blackModels=['combin', 'worst', 'ideal'],
    noSubsampling=True,
    doNormalization=True,
)

In [ ]:
bests2 = twinewsGet\
(
    splitVersion=2,
    onlyBestForField='ndcg',
    blackModels=['combin', 'worst', 'ideal'],
    noSubsampling=True,
    doNormalization=True,
)

In [ ]:
combins2 = twinewsGet\
(
    splitVersion=2,
    whiteModels=['combin'],
    noSubsampling=True,
    doNormalization=True,
)

In [ ]:
alonesOthers = twinewsGet\
(
    splitVersion=splitVersion,
    blackModels=['bm25', 'dbert-ft', 'combin', 'random', 'ideal', 'worst'],
    noSubsampling=True,
    onlyBestForField='ndcg',
    doNormalization=doNormalization,
)

# Displaying data

In [8]:
def displayRows(rows):
    displayDicts([dictSelect(e, ['id', 'ndcg', 'maxUsers', 'splitVersion', 'div@100']) for e in rows])

In [9]:
displayRows(alonesOthers)

NameError: name 'alonesOthers' is not defined

# Multi-objective of combins

In [10]:
def getFigure():
    global xaxis, yaxis, xrange, yrange, splitVersion, doNormalization, palette, blackIds, paretoFrontier, addTitle, displayAlones, hideTickLabels, hideGrid, margeRatio, hideXLabel, hideYLabel, hideTickLines
    rows = twinewsGet\
    (
        splitVersion=splitVersion,
        whiteModels=['combin'],
        noSubsampling=True,
        doNormalization=doNormalization,
    )
    rows = [row for row in rows if 'rankAsScore' not in row or row['rankAsScore'] == [False, False]]
    # rows = [row for row in rows if row['model'] != 'combin' or not isModels(row, "stylo", "word2vec")]
    rows = [row for row in rows if row['id'] not in blackIds]
    # bp(rows)
    # print(len(rows))
    alones = twinewsGet\
    (
        splitVersion=splitVersion,
        whiteModels=['bm25', 'dbert-ft'],
        noSubsampling=True,
        onlyBestForField='ndcg',
        doNormalization=doNormalization,
    )
    alonesOthers = twinewsGet\
    (
        splitVersion=splitVersion,
        blackModels=['bm25', 'dbert-ft', 'combin', 'random', 'ideal', 'worst'],
        noSubsampling=True,
        onlyBestForField='ndcg',
        doNormalization=doNormalization,
    )
    bm25 = alones[0] if 'bm25' in alones[0]['id'] else alones[1]
    dbertft = alones[0] if 'dbert-ft' in alones[0]['id'] else alones[1]
    # bp(bm25)
    # bp(dbertft)
    havingBoth = {'x': [], 'y': [], 'id': [], 'model': []}
    havingDBertft = copy.deepcopy(havingBoth)
    havingBm25 = copy.deepcopy(havingBoth)
    others = copy.deepcopy(havingBoth)
    for row in rows:
        modelsRepr = str(row['models']) if 'models' in row else ""
        model = getModel(row)
        if 'dbert-ft' in modelsRepr and 'bm25' in modelsRepr:
            havingBoth['x'].append(row[xaxis])
            havingBoth['y'].append(row[yaxis])
            havingBoth['model'].append(model)
            havingBoth['id'].append(row['id'])
        elif 'dbert-ft' in modelsRepr:
            havingDBertft['x'].append(row[xaxis])
            havingDBertft['y'].append(row[yaxis])
            havingDBertft['model'].append(model)
            havingDBertft['id'].append(row['id'])
        elif 'bm25' in modelsRepr:
            havingBm25['x'].append(row[xaxis])
            havingBm25['y'].append(row[yaxis])
            havingBm25['model'].append(model)
            havingBm25['id'].append(row['id'])
        else:
            others['x'].append(row[xaxis])
            others['y'].append(row[yaxis])
            others['model'].append(model)
            others['id'].append(row['id'])
    # bp([havingBoth, havingDBertft, havingBm25, others])
    # A function to search for the right row:
    def getRow(rows, id):
        for row in rows:
            if row['id'] == id:
                return row
    if width <= 500:
        row = getRow(rows + alones, paretoFrontier[0])
        newYTop = row[yaxis] + margeRatio * abs(yrange[0] - yrange[1])
        yrange = (yrange[0], newYTop)
        row = getRow(rows + alones, paretoFrontier[-1])
        newXRight = row[xaxis] + margeRatio * abs(xrange[0] - xrange[1])
        xrange = (xrange[0], newXRight)
    TOOLTIPS = [("model", "@model"), ("id", "@id")]
    rangeKwargs = {} if xrange is None else {'x_range': xrange, 'y_range': yrange}
    title = "Multi-objective (" + xaxis + " and " + yaxis + ") of combinations"
    if width <= 300:
        title = yaxis + " a.a.f.o. " + xaxis # "as a function of", "given" + "a.a.f.o."
    x_axis_label = None if hideXLabel else xaxis
    y_axis_label = None if hideYLabel else yaxis
    if not addTitle:
        title = None
    p = figure(title=title, x_axis_label=x_axis_label, y_axis_label=y_axis_label, tooltips=TOOLTIPS, width=width, height=height, **rangeKwargs)
    # We define styles:
    styles = \
    {
        "combination": (p.circle, palette[3], circleSizes[2]),
        "havingBoth": (p.diamond, palette[0], diamondSizes[0]),
        "havingDBertft": (p.triangle, palette[1], triangleSizes[1]),
        "havingBm25": (p.square, palette[0], squareSizes[1]),
        "bm25": (p.square, palette[5], squareSizes[1]),
        "dbertft": (p.triangle, palette[5], triangleSizes[0]),
        "alone": (p.circle, palette[5], circleSizes[2]),
        "pareto": (p.line, palette[4], lineWidth),
    }
    for key in styles:
        styles[key] = {"shape": styles[key][0], "color": styles[key][1], "size": styles[key][2]}
    # Misc params:
    lineAlpha = 0.4
    lineDash = 'dashed' # dashed, dotted, dotdash, dashdot
    # We draw the pareto frontier:
    x, y = [], []
    for current in paretoFrontier:
        for row in rows + alones:
            if row['id'] == current:
                x.append(row[xaxis])
                y.append(row[yaxis])
                break
    styles['pareto']['shape'](x, y, line_width=styles['pareto']['size'], line_color=styles['pareto']['color'], line_dash=lineDash)
    # We draw the left extension:
    row = getRow(rows + alones, paretoFrontier[0])
    x = [xrange[0], row[xaxis]]
    y = [row[yaxis], row[yaxis]]
    styles['pareto']['shape'](x, y, line_width=styles['pareto']['size'], line_color=styles['pareto']['color'], line_dash=lineDash, line_alpha=lineAlpha)
    # We draw the bottom extension:
    row = getRow(rows + alones, paretoFrontier[-1])
    x = [row[xaxis], row[xaxis]]
    y = [row[yaxis], 0]
    styles['pareto']['shape'](x, y, line_width=styles['pareto']['size'], line_color=styles['pareto']['color'], line_dash=lineDash, line_alpha=lineAlpha)
    # We draw all combinations:
    args = ('x', 'y')
    sizeKwargs = {'size': styles['combination']['size']}
    styles['combination']['shape'](*args, **sizeKwargs, source=ColumnDataSource(others), **getColorKwargs(styles['combination']['color']))
    sizeKwargs = {'size': styles['havingBoth']['size']}
    styles['havingBoth']['shape'](*args, **sizeKwargs, source=ColumnDataSource(havingBoth), **getColorKwargs(styles['havingBoth']['color']))
    sizeKwargs = {'size': styles['havingDBertft']['size']}
    styles['havingDBertft']['shape'](*args, **sizeKwargs, source=ColumnDataSource(havingDBertft), **getColorKwargs(styles['havingDBertft']['color']))
    sizeKwargs = {'size': styles['havingBm25']['size']}
    styles['havingBm25']['shape'](*args, **sizeKwargs, source=ColumnDataSource(havingBm25), **getColorKwargs(styles['havingBm25']['color']))
    # Adding models alones:
    bm25Source = {'x': [bm25[xaxis]], 'y': [bm25[yaxis]], 'model': [bm25['model']], 'id': [bm25['id']]}
    dbertftSource = {'x': [dbertft[xaxis]], 'y': [dbertft[yaxis]], 'model': [dbertft['model']], 'id': [dbertft['id']]}
    sizeKwargs = {'size': styles['bm25']['size']}
    styles['bm25']['shape'](*args, source=bm25Source, **sizeKwargs, **getColorKwargs(styles['bm25']['color']))
    sizeKwargs = {'size': styles['dbertft']['size']}
    styles['dbertft']['shape'](*args, source=dbertftSource, **sizeKwargs, **getColorKwargs(styles['dbertft']['color']))
    # Adding other models being alone:
    source = {'x': [e[xaxis] for e in alonesOthers], 'y': [e[yaxis] for e in alonesOthers], 'model': [e['model'] for e in alonesOthers], 'id': [e['id'] for e in alonesOthers]}
    sizeKwargs = {'size': styles['alone']['size']}
    if displayAlones:
        styles['alone']['shape'](*args, source=source, **sizeKwargs, **getColorKwargs(styles['alone']['color']))
    # Adding the legend:
    if displayLegend:
        legends = []
        legendTexts = \
        {
            # 'pareto': "Pareto frontier",
            # 'comb': "Combination",
            # 'both': "Having dbert-ft and bm25",
            # 'comb-dbert-ft': "Having at least dbert-ft",
            # 'comb-bm25': "Having at least bm25",
            # 'alone-dbert': "dbert-ft alone",
            # 'alone-bm25': "bm25 alone",
            # 'alone': "Alone model",
            'pareto': "Pareto frontier",
            'comb': "Other combinations of models",
            'both': "Combination of dbert-ft and bm25",
            'comb-dbert-ft': "Combinations having dbert-ft at least",
            'comb-bm25': "Combinations having bm25 at least",
            'alone-dbert': "dbert-ft alone",
            'alone-bm25': "bm25 alone",
            'alone': "Other lone models",
        }
        legends.append(LegendItem(label=legendTexts['pareto'], renderers=[p.renderers[0]]))
        legends.append(LegendItem(label=legendTexts['comb'], renderers=[p.renderers[1 + 2]]))
        legends.append(LegendItem(label=legendTexts['both'], renderers=[p.renderers[2 + 2]]))
        legends.append(LegendItem(label=legendTexts['comb-dbert-ft'], renderers=[p.renderers[3 + 2]]))
        legends.append(LegendItem(label=legendTexts['comb-bm25'], renderers=[p.renderers[4 + 2]]))
        legends.append(LegendItem(label=legendTexts['alone-bm25'], renderers=[p.renderers[5 + 2]]))
        legends.append(LegendItem(label=legendTexts['alone-dbert'], renderers=[p.renderers[6 + 2]]))
        if displayAlones:
            legends.append(LegendItem(label=legendTexts['alone'], renderers=[p.renderers[7 + 2]]))
        if displayAlones:
            # legendsOrder = \
            # [
            #     legends[1], legends[3], legends[4],
            #     legends[2], legends[7], legends[6],
            #     legends[5], legends[0],
            # ]
            legendsOrder = \
            [
                legends[3], legends[4], legends[2],
                legends[1], legends[6], legends[5],
                legends[7], legends[0],
            ]
        else:
            legendsOrder = \
            [
                legends[1], legends[3], legends[4],
                legends[2], legends[6],
                legends[5], legends[0],
            ]
        legend1 = Legend(items=legendsOrder, location='bottom_right' if 'ser' in xaxis else 'top_right')
        p.add_layout(legend1)
    if hideTickLabels:
        p.xaxis.major_label_text_font_size = '0pt'
        p.yaxis.major_label_text_font_size = '0pt'
    if hideGrid:
        p.xgrid.grid_line_color = None
        p.ygrid.grid_line_color = None
    # https://docs.bokeh.org/en/latest/docs/reference/models/axes.html
    if hideTickLines:
        p.xaxis.minor_tick_line_width = 0.0
        p.xaxis.major_tick_line_width = 0.0
        p.yaxis.minor_tick_line_width = 0.0
        p.yaxis.major_tick_line_width = 0.0
    return p

In [11]:
# To save all figures:
figures = dict()

In [12]:
# Misc vars:
path = '/home/hayj/Dashboard/Articles/Manuscrit/MultiObjective/sources'
width = 600 # 600, 250, 300 (300 si on met tous les tick et labels)
height = width
displayAlones = True
margeRatio = 0.06
if width == 600:
    circleSizes = [15, 12, 10]
    triangleSizes = [15, 12, 10]
    diamondSizes = [18, 15, 12]
    squareSizes = [13, 10, 8]
    lineWidth = 3
    displayLegend = True
    addTitle = False
    hideTickLabels = False
    hideXLabel = False
    hideYLabel = False
    hideTickLines = False
elif width <= 300:
    circleSizes = [8, 8, 6]
    triangleSizes = [11, 11, 8]
    diamondSizes = [13, 10, 9]
    squareSizes = [8, 8, 5]
    lineWidth = 2
    displayLegend = False
    addTitle = False
    hideTickLabels = False
    hideXLabel = False
    hideYLabel = False
    hideTickLines = False
else:
    print("ERROR")
hideGrid = hideTickLabels

In [13]:
# Diversity:
xaxis = 'div@100'
yaxis = 'ndcg'
xrange, yrange = (0.45, 0.65), (0.45, 0.70)
splitVersion = 1
doNormalization = True
palette = palettes[2]
blackIds = {'combin-d2d96', 'combin-904f0'}
paretoFrontier = ['combin-8f846', 'combin-ea42e', 'combin-1ca06', 'combin-bbe73', 'dbert-ft-7847a', 'combin-093a2', 'combin-4f0a5']
p = getFigure()
figures[xaxis] = p
show(p)
exportAsSVG(p, path + "/" + xaxis + ".svg")

Normalized metrics: {'topic-div@100', 'div@100', 'style-nov@100', 'topic-snov@100', 'snov@100', 'style-div@100', 'nov@100', 'style-snov@100', 'topic-nov@100'}
Normalized metrics: {'topic-div@100', 'div@100', 'style-nov@100', 'topic-snov@100', 'snov@100', 'style-div@100', 'nov@100', 'style-snov@100', 'topic-nov@100'}
Normalized metrics: {'topic-div@100', 'div@100', 'style-nov@100', 'topic-snov@100', 'snov@100', 'style-div@100', 'nov@100', 'style-snov@100', 'topic-nov@100'}


NameError: name 'exportAsSVG' is not defined

In [ ]:
# Topic diversity:
xaxis = 'topic-div@100'
yaxis = 'ndcg'
xrange, yrange = (0.25, 0.51), (0.39, 0.715) # Ou moins bien (0.225, 0.525), (0.39, 0.69)
# xrange, yrange = (0, 1), (0, 1)
splitVersion = 1
doNormalization = True
palette = palettes[2]
blackIds = {'combin-d2d96', 'combin-904f0'}
paretoFrontier = ['combin-8f846', 'combin-6ecf3', 'combin-d0fd0', 'combin-80112', 'combin-c12e9', 'dbert-ft-7847a']
p = getFigure()
figures[xaxis] = p
show(p)
exportAsSVG(p, path + "/" + xaxis + ".svg")

In [ ]:
# Jacccard diversity:
xaxis = 'jacc-div@100'
yaxis = 'ndcg'
xrange, yrange = (0.855, 0.885), (0.35, 0.65)
splitVersion = 1
doNormalization = True
palette = palettes[2]
blackIds = {'combin-d2d96', 'combin-904f0'}
paretoFrontier = ['combin-8f846', 'combin-4f0a5']
p = getFigure()
figures[xaxis] = p
show(p)
exportAsSVG(p, path + "/" + xaxis + ".svg")

In [ ]:
# SW Jacccard diversity:
xaxis = 'swjacc-div@100'
yaxis = 'ndcg'
xrange, yrange = (0.945, 0.973), (0.4, 0.65)
splitVersion = 1
doNormalization = True
palette = palettes[2]
blackIds = {'combin-d2d96', 'combin-904f0'}
paretoFrontier = ['combin-8f846', 'combin-4f0a5']
p = getFigure()
figures[xaxis] = p
show(p)
exportAsSVG(p, path + "/" + xaxis + ".svg")

In [ ]:
# Style diversity:
xaxis = 'style-div@100'
yaxis = 'ndcg'
xrange, yrange = (0.1, 0.5), (0.25, 0.65)
splitVersion = 1
doNormalization = True
palette = palettes[2]
blackIds = {'combin-d2d96', 'combin-904f0'}
paretoFrontier = ['combin-8f846', 'combin-4f0a5']
p = getFigure()
figures[xaxis] = p
show(p)
exportAsSVG(p, path + "/" + xaxis + ".svg")

In [ ]:
# Novelty:
xaxis = 'nov@100'
yaxis = 'ndcg'
xrange, yrange = (0.49, 0.62), (0.49, 0.67)
splitVersion = 1
doNormalization = True
palette = palettes[2]
blackIds = {'combin-d2d96', 'combin-904f0'}
paretoFrontier = ['combin-8f846', 'combin-d0fd0', 'combin-1ca06', 'combin-c12e9', 'dbert-ft-7847a']
p = getFigure()
figures[xaxis] = p
show(p)
exportAsSVG(p, path + "/" + xaxis + ".svg")

In [ ]:
# Topic novelty:
xaxis = 'topic-nov@100'
yaxis = 'ndcg'
xrange, yrange = (0.215, 0.445), (0.45, 0.68)
splitVersion = 1
doNormalization = True
palette = palettes[2]
blackIds = {'combin-d2d96', 'combin-904f0'}
paretoFrontier = ['combin-8f846', 'combin-6ecf3', 'combin-d0fd0', 'combin-1ca06', 'combin-80112', 'combin-c12e9', 'dbert-ft-7847a', 'combin-8169e']
p = getFigure()
figures[xaxis] = p
show(p)
exportAsSVG(p, path + "/" + xaxis + ".svg")

In [17]:
# SW Jaccard novelty:
xaxis = 'jacc-nov@100'
yaxis = 'ndcg'
xrange, yrange = (0.215, 0.445), (0.45, 0.68)
splitVersion = 1
doNormalization = True
palette = palettes[2]
blackIds = {'combin-d2d96', 'combin-904f0'}
paretoFrontier = ['combin-8f846', 'combin-6ecf3', 'combin-d0fd0', 'combin-1ca06', 'combin-80112', 'combin-c12e9', 'dbert-ft-7847a', 'combin-8169e']
p = getFigure()
figures[xaxis] = p
show(p)
exportAsSVG(p, path + "/" + xaxis + ".svg")

In [ ]:
# Strict novelty:
xaxis = 'snov@100'
yaxis = 'ndcg'
xrange, yrange = (0.409, 0.56), (0.47, 0.67)
splitVersion = 1
doNormalization = True
palette = palettes[2]
blackIds = {'combin-d2d96', 'combin-904f0'}
paretoFrontier = ['combin-8f846', 'combin-d0fd0', 'combin-ea42e', 'combin-1ca06', 'combin-80112', 'combin-c12e9', 'dbert-ft-7847a']
p = getFigure()
figures[xaxis] = p
show(p)
exportAsSVG(p, path + "/" + xaxis + ".svg")

In [ ]:
# Topic strict novelty:
xaxis = 'topic-snov@100'
yaxis = 'ndcg'
xrange, yrange = (0.25, 0.54), (0.41, 0.68)
splitVersion = 1
doNormalization = True
palette = palettes[2]
blackIds = {'combin-d2d96', 'combin-904f0'}
paretoFrontier = ['combin-8f846', 'combin-6ecf3', 'combin-d0fd0', 'combin-1ca06', 'combin-80112', 'combin-c12e9', 'combin-71538', 'dbert-ft-7847a', 'combin-8169e']
p = getFigure()
figures[xaxis] = p
show(p)
exportAsSVG(p, path + "/" + xaxis + ".svg")

In [ ]:
# swjacc-snov@100, swjacc-nov@100 aucun interet

In [14]:
# SW Jaccard strict novelty:
xaxis = 'swjacc-snov@100'
yaxis = 'ndcg'
xrange, yrange = (0.215, 0.445), (0.45, 0.68)
splitVersion = 1
doNormalization = True
palette = palettes[2]
blackIds = {'combin-d2d96', 'combin-904f0'}
paretoFrontier = ['combin-8f846', 'combin-6ecf3', 'combin-d0fd0', 'combin-1ca06', 'combin-80112', 'combin-c12e9', 'dbert-ft-7847a', 'combin-8169e']
p = getFigure()
figures[xaxis] = p
show(p)
exportAsSVG(p, path + "/" + xaxis + ".svg")

NameError: name 'exportAsSVG' is not defined

In [ ]:
# Serendipity:
xaxis = 'avg-ser@100' # {'tfidf-ser@100', 'wtfidf-ser@100'}
yaxis = 'ndcg'
xrange, yrange = (0.14, 0.46), (0.42, 0.65)
splitVersion = 1
doNormalization = True
palette = palettes[2]
blackIds = {'combin-d2d96', 'combin-904f0'}
paretoFrontier = ['combin-8f846', 'combin-d0fd0', 'combin-ea42e']
p = getFigure()
figures[xaxis] = p
show(p)
export_png(p, filename=path + "/" + xaxis + ".png")
exportAsSVG(p, path + "/" + xaxis + ".svg")

In [ ]:
# Serendipity:
xaxis = 'jacc-ser@100'
yaxis = 'ndcg'
xrange, yrange = (0.5, 0.7), (0.45, 0.65)
splitVersion = 1
doNormalization = True
palette = palettes[2]
blackIds = {'combin-d2d96', 'combin-904f0'}
paretoFrontier = ['combin-8f846', 'combin-d0fd0', 'combin-ea42e']
p = getFigure()
figures[xaxis] = p
show(p)
export_png(p, filename=path + "/" + xaxis + ".png")
exportAsSVG(p, path + "/" + xaxis + ".svg")

In [ ]:
# Serendipity:
xaxis = 'bm25-ser@100'
yaxis = 'ndcg'
xrange, yrange = (0.0, 0.4), (0.35, 0.65)
splitVersion = 1
doNormalization = True
palette = palettes[2]
blackIds = {'combin-d2d96', 'combin-904f0'}
paretoFrontier = ['combin-8f846', 'combin-6ecf3', 'combin-d0fd0', 'combin-ea42e', 'dbert-ft-7847a']
p = getFigure()
figures[xaxis] = p
show(p)
export_png(p, filename=path + "/" + xaxis + ".png")
exportAsSVG(p, path + "/" + xaxis + ".svg")

In [ ]:
# Serendipity:
xaxis = 'style-ser@100'
yaxis = 'ndcg'
xrange, yrange = (0.0, 0.6), (0.35, 0.65)
splitVersion = 1
doNormalization = True
palette = palettes[2]
blackIds = {'combin-d2d96', 'combin-904f0'}
paretoFrontier = ['combin-8f846', 'combin-d0fd0', 'combin-ea42e']
p = getFigure()
figures[xaxis] = p
show(p)
export_png(p, filename=path + "/" + xaxis + ".png")
exportAsSVG(p, path + "/" + xaxis + ".svg")

In [ ]:
# Grid:
grid = gridplot\
(
    [
        [figures['div@100'], figures['topic-div@100']],
        [figures['nov@100'], figures['topic-nov@100']],
        [figures['snov@100'], figures['topic-snov@100']],
    ],
    # plot_width=250,
    # plot_height=250
)
show(grid)